<a href="https://colab.research.google.com/github/deejay14/Jhakas-NewsWala/blob/main/Users_vs_artciles_final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [180]:
df=pd.read_csv('gdrive/My Drive/final_fixed_scrapped.csv')

In [181]:
def user_vs_article(df,users_no): ## df1 - dataframe of doc vs topic, no. of users
    import secrets
    from sklearn.mixture import GaussianMixture
    #df=pd.read_csv('gdrive/My Drive/final_fixed_scrapped.csv')
    df1=pd.read_csv('gdrive/My Drive/final scrapped/doc_vs_topic.csv')
    df2=pd.read_csv('gdrive/My Drive/final scrapped/topicwise_indices.csv')
    list=df['totalwords'].tolist()
    time=[]
    for i in range(7843):
        time.append(round((60*(list[i]/200)),2))
    df['average_reading_time(sec)']=time
    counts=df['average_reading_time(sec)'].value_counts()
    data =np.column_stack((counts.axes[0].tolist(),counts.tolist()))
 
# training gaussian mixture model 
    
    gmm = GaussianMixture(n_components=12)
    model=gmm.fit(data)

    topic0=df2.loc[0].tolist()[1:121]    
    topic2=df2.loc[1].tolist()[1:2746]
    topic3=df2.loc[2].tolist()[1:1000]
    topic4=df2.loc[3].tolist()[1:1222]
    topic5=df2.loc[4].tolist()[1:34]
    topic7=df2.loc[5].tolist()[1:2463]
    topic10=df2.loc[6].tolist()[1:264]
    topic0=[int(x) for x in topic0]
    topic2=[int(x) for x in topic2]
    topic3=[int(x) for x in topic3]
    topic4=[int(x) for x in topic4]
    topic5=[int(x) for x in topic5]
    topic7=[int(x) for x in topic7]
    topic10=[int(x) for x in topic10]
     


    ## Clickstream generator lists
    lz=[]## reference list
    for i in range(users_no):
        flag = True
        while flag:
              s = np.random.poisson(4, 1)
              if s[0]> 0:
                 lz.append(s[0])
                 flag=False
        
              else: flag=True
    user_id=[]
    #print(lz)
    k=1
    for i in range(len(lz)):
        for j in range(lz[i]*10):
            user_id.append(k)
        k=k+1
    rank=[]
    t=[1,2,3,4,5,6,7,8,9,10]
    for i in range(len(lz)):
        for j in range(lz[i]):
            rank=rank+t  


    q=[]## click list
    for i in range(len(lz)):
        for j in range(lz[i]):
           r=np.random.poisson(5, 1)
           t=np.random.binomial(1,0.5,size=10)
           min=np.minimum(r,sum(t))
            #print(min)
           j=0
           a=0
           while j < min:
                if t[a]==1:
                   j=j+1
                   a=a+1
                else: a=a+1
 
           t_=t[0:a]
           t_=t_.tolist()

           for i in range(10-a):
               t_=t_ + [0]
           q=q+t_
    click=[]
    read_time=[]
    for i in range(len(q)):
       if q[i]==0:
          click.append('No')
       else: click.append('Yes')
    for i in range(len(q)):
        if q[i]==0:
           read_time.append('NaN')
        else: read_time.append(abs(model.sample(1)[0][0][0]))            
#session id
    sessionid=[]

    for j in lz:
        for k in range(j):
            t=k+1
            t_=[t for i in range(10)]
            sessionid=sessionid+t_  
    article_id=[]
    for i in range(len(lz)):
       for j in range(lz[i]):
          for p1 in range(2):
              article_id.append(secrets.choice(topic2))
          for p2 in range(2):
              article_id.append(secrets.choice(topic7))
          for p3 in range(2):
              article_id.append(secrets.choice(topic4))    
          article_id.append(secrets.choice(topic3))
          article_id.append(secrets.choice(topic10))
          article_id.append(secrets.choice(topic0))
          article_id.append(secrets.choice(topic5)) 

    t_spent=read_time
    article_id_list=article_id 
    #print(t_spent)
    user_vs_doc=np.zeros((users_no,7843)) 



    ## User vs article matrix generator
    u=0
    j0=0
    #print(lz)
    for i in lz:  
        for j in range(i*10):
            j0=sum(lz[0:u])*10+j
            #print(j0)
            index=article_id_list[j0]
            if type(t_spent[j0])==str:
               user_vs_doc[u][index]=0.0
          #print(index)  
            else: user_vs_doc[u][index] = t_spent[j0]/time[index]
        #print(user_vs_doc[u][index])
        u=u+1
    for i in range(users_no):
        for j in range(7843):
            if user_vs_doc[i][j]>1:
               user_vs_doc[i][j]=1.0
    #return(user_vs_doc)

    columns=[i for i in range(7843)]   
    index=[j for j in range(users_no)]
    df_user_vs_doc = pd.DataFrame(user_vs_doc, columns =columns, index = index) 
    return(df_user_vs_doc)      
















      


In [183]:
p=user_vs_article(df,1000)

In [184]:
p.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7803,7804,7805,7806,7807,7808,7809,7810,7811,7812,7813,7814,7815,7816,7817,7818,7819,7820,7821,7822,7823,7824,7825,7826,7827,7828,7829,7830,7831,7832,7833,7834,7835,7836,7837,7838,7839,7840,7841,7842
count,1000.0,1000.0,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.0,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000
mean,0.0,0.0,0.001000,0.003000,0.003994,0.0,0.000536,0.003361,0.004205,0.002000,0.0,0.002000,0.000839,0.0,0.000431,0.001165,0.002985,0.001264,0.001456,0.001392,0.0,0.001486,0.0,0.001000,0.0,0.002000,0.002000,0.005134,0.0,0.003275,0.0,0.0,0.001374,0.000797,0.001933,0.003642,0.001000,0.002000,0.001000,0.003000,...,0.001000,0.000719,0.001000,0.001000,0.001733,0.002423,0.001000,0.001000,0.0,0.000405,0.002000,0.002545,0.001000,0.0,0.003000,0.000744,0.002591,0.002000,0.0,0.002000,0.002000,0.001000,0.002000,0.000727,0.024595,0.0,0.002751,0.026419,0.021870,0.002000,0.001000,0.001000,0.001568,0.000400,0.001000,0.002731,0.001000,0.001000,0.0,0.002000
std,0.0,0.0,0.031623,0.054717,0.059390,0.0,0.016947,0.055874,0.061452,0.044699,0.0,0.044699,0.026542,0.0,0.013626,0.026371,0.048134,0.025490,0.032557,0.027831,0.0,0.029928,0.0,0.031623,0.0,0.044699,0.044699,0.068044,0.0,0.053916,0.0,0.0,0.033753,0.025203,0.043205,0.049214,0.031623,0.044699,0.031623,0.054717,...,0.031623,0.022721,0.031623,0.031623,0.039192,0.046638,0.031623,0.031623,0.0,0.012801,0.044699,0.047880,0.031623,0.0,0.054717,0.023513,0.048427,0.044699,0.0,0.044699,0.044699,0.031623,0.044699,0.022975,0.152349,0.0,0.050588,0.151855,0.141674,0.044699,0.031623,0.031623,0.036357,0.012647,0.031623,0.050293,0.031623,0.031623,0.0,0.044699
min,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00